In [1]:
import numpy as np
from stable_baselines3 import PPO
# from sb3_contrib import TRPO
from gymnasium import spaces, Env
from boat_simulation import Boat, wrap_phase
from environments import MultiMarkNoModelEnv

2024-04-07 23:26:27.949563: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 23:26:27.967854: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 23:26:27.967872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 23:26:27.968381: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-07 23:26:27.971759: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import torch
import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor


def linear(device, x, in_units, out_units, activation):
    x = nn.Linear(in_units, out_units).to(device)(x)
    return activation().to(device)(x)

def mlp(device, x, units):
    for i in range(0, len(units)):
        if not isinstance(units[i], tuple):
            units[i] = (units[i], nn.ReLU)

    for i in range(1, len(units)):
        prev_layer_size, _ = units[i - 1]
        layer_size, activation = units[i]
        x = linear(device, x, prev_layer_size, layer_size, activation)
    return x

def self_attention(device, x, num_heads, num_layers):
    _batch, seq_len, embed_size = x.shape

    attn_mod = nn.MultiheadAttention(embed_size, num_heads, dropout=0.5)
    for i in range(1, num_layers):
        lin = linear(device, x, embed_size, embed_size, nn.ReLU)
        attn,  _weights = attn_mod.to(device)(lin, lin, lin, need_weights=False)
        x = attn + x
    return x


class CustomMLP(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=1):
        super(CustomMLP, self).__init__(observation_space, features_dim)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def forward(self, observations):
        x = observations.to(self.device)
        return mlp(self.device, x, [7, 400, 300, (self.features_dim, nn.Tanh)])


# Define the LSTM model
class NonLSTM(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim):
        super(NonLSTM, self).__init__(observation_space, features_dim)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.hidden_layer_size = 64
        x1, x2 = observation_space.shape
        input_size = x1 * x2
        self.linear = lambda x: mlp(self.device, x, [input_size, self.hidden_layer_size, self.hidden_layer_size, features_dim])

    def forward(self, input_seq):
        batch_size = input_seq.shape[0]
        input = input_seq.view(batch_size, -1)
        predictions = self.linear(input)
        return predictions

class SelfAttentionExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim):
        super(SelfAttentionExtractor, self).__init__(observation_space, features_dim)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        (seq_size, input_size,) = observation_space.shape
        self.attention = lambda x: self_attention(self.device, x, 2, 4)
        # self.input_layer = lambda x: mlp(self.device, x, [input_size, features_dim])
        self.output_layer = lambda x: mlp(self.device, x, [seq_size * input_size, features_dim])

    def forward(self, input_seq):
        input = input_seq.to(self.device)
        hidden = self.attention(input)
        hidden_flat = nn.Flatten().to(self.device)(hidden)
        output = self.output_layer(hidden_flat)
        return output

In [3]:
from IPython.core.display import display, HTML, Javascript

disable_scroll_script = """
var idx = Jupyter.notebook.get_selected_index();
var cell = Jupyter.notebook.get_cell(idx);
cell.output_area._should_scroll = function(lines) {
    return false;
}
"""

display(Javascript(disable_scroll_script))

/tmp/ipykernel_359240/738611061.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, Javascript


<IPython.core.display.Javascript object>

In [4]:
import plotly.graph_objects as go
import time
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HBox

outer_radius = 2 * 250 + 2.5 * 0.1 * 250

dt = 1
num_marks = 1
max_seconds = 500 * num_marks / dt

# Initialize the figure and scatter plot
fig = go.FigureWidget()
scatter = fig.add_scatter(mode='markers+lines', name='trajectory')
marks_scatter = fig.add_scatter(mode='markers+text', name='marks')
fig.update_xaxes(range=[-outer_radius, outer_radius],dtick=25)
fig.update_yaxes(range=[-outer_radius, outer_radius],dtick=25)
fig.layout.width=800
fig.layout.height=800


reward_fig = go.FigureWidget()
reward_scatter = reward_fig.add_scatter(mode='markers+lines', name='reward')
reward_fig.update_xaxes(range=[0, max_seconds],dtick=25)
reward_fig.update_yaxes(range=[-2, 2],dtick=0.2)
reward_fig.layout.width=800
reward_fig.layout.height=600
reward_fig.layout.title = "Reward over time"

heading_fig = go.FigureWidget()
heading_scatter = heading_fig.add_scatter(mode='markers+lines', name='heading')
heading_fig.update_xaxes(range=[0, max_seconds],dtick=25)
heading_fig.update_yaxes(range=[-210, 210],dtick=30)
heading_fig.layout.width=800
heading_fig.layout.height=600
heading_fig.layout.title = "Heading over time"

vmg_fig = go.FigureWidget()
vmg_scatter = vmg_fig.add_scatter(mode='markers+lines')
vmg_fig.update_xaxes(range=[0, max_seconds],dtick=25)
vmg_fig.update_yaxes(range=[-4, 4],dtick=2)
vmg_fig.layout.width=800
vmg_fig.layout.height=400
vmg_fig.layout.title = "VMG over time"

distance_fig = go.FigureWidget()
distance_fig.add_scatter(mode='markers+lines')
distance_fig.add_scatter(mode='markers+lines')
distance_fig.update_xaxes(range=[0, max_seconds],dtick=25)
distance_fig.update_yaxes(type='log', range=[0, 3], dtick=1)
distance_fig.layout.width=800
distance_fig.layout.height=400
distance_fig.layout.title = "Distance over time"

# Initialize output for the text
out = widgets.Output()

# Create horizontal boxes to hold the figures
box1 = HBox([fig, reward_fig])
box2 = HBox([heading_fig, vmg_fig, distance_fig])

# Display text and figures
display(out, box1, box2)

episode = 0

colormap = ['blue', 'red', 'green', 'orange', 'black']

def plot(data, marks, bounds):
    global episode, out, fig
    episode += 1
    if episode % 50 != 0:
        return
    x_values = [point['x'] for point in data]
    y_values = [point['y'] for point in data]
    meta_values = [{k: '%.3f' % v for (k, v) in point['meta'].items()} for point in data]

    min_x, max_x, min_y, max_y = bounds


    color_meta = [point['meta']['current_mark'] for point in data]
    colors = [colormap[color % len(colormap)] for color in color_meta]

    headings = [point['meta']['heading'] for point in data]
    vmg = [point['meta']['vmg'] for point in data]
    rewards = [point['meta']['reward'] for point in data]
    distances = [point['meta']['distance'] for point in data]
    min_distances = [point['meta']['min_distance'] for point in data]

    global_meta = {'final_position': (x_values[-1], y_values[-1]), 'reward': sum(rewards), 'iters': len(data), 'episode': episode}

    # Update data
    scatter = fig.data[0]
    scatter.x = x_values
    scatter.y = y_values
    scatter.hovertext = meta_values  # Add this line
    scatter.marker.color = colors
    scatter.line.color = 'lightgrey'

    marks_scatter = fig.data[1]
    marks_scatter.x = [x for x, _ in marks]
    marks_scatter.y = [y for _, y in marks]
    marks_scatter.text = list(range(len(marks)))
    marks_scatter.textposition = 'bottom right'
    marks_scatter.marker.color = 'black'
    marks_scatter.marker.symbol = 'x'
    marks_scatter.marker.size = 10

    heading_scatter = heading_fig.data[0]
    heading_scatter.x = list(range(len(headings)))
    heading_scatter.y = headings
    heading_scatter.marker.color = colors
    heading_scatter.line.color = 'lightgrey'

    reward_scatter = reward_fig.data[0]
    reward_scatter.x = list(range(len(rewards)))
    reward_scatter.y = rewards
    reward_scatter.marker.color = colors
    reward_scatter.line.color = 'lightgrey'

    vmg_scatter = vmg_fig.data[0]
    vmg_scatter.x = list(range(len(vmg)))
    vmg_scatter.y = vmg
    vmg_scatter.marker.color = colors
    vmg_scatter.line.color = 'lightgrey'

    distance_scatter = distance_fig.data[0]
    distance_scatter.x = list(range(len(distances)))
    distance_scatter.y = distances
    distance_scatter.marker.color = 'blue'
    distance_scatter.line.color = 'lightgrey'

    min_distance_scatter = distance_fig.data[1]
    min_distance_scatter.x = list(range(len(min_distances)))
    min_distance_scatter.y = min_distances
    min_distance_scatter.marker.color = 'green'
    min_distance_scatter.line.color = 'lightgrey'

    fig.update_xaxes(range=[min_x * 1.1, max_x * 1.1],dtick=25)
    fig.update_yaxes(range=[min_y * 1.1, max_y * 1.1],dtick=25)
    fig.layout.width=800
    fig.layout.height=800

    heading_fig.update_yaxes(range=[-210, 210],dtick=30)
    heading_fig.layout.width=800
    heading_fig.layout.height=600

    vmg_fig.layout.width=800
    vmg_fig.layout.height=400

    # Update the text output
    with out:
        out.clear_output(wait=True)
        print(f"Global Meta: {str(global_meta)}")

None

Output()

    'data': [{'mode': 'markers+lines',
              'name': 'trajectory',
     …

    'data': [{'mode': 'markers+lines',
              'name': 'heading',
        …

In [5]:
# Configuration

r = 250
actions = np.array([-5, -1, 0, 1, 5]) / 180.0

config = {
    'max_marks': 1,
    'max_seconds_per_leg': 500,
    'plot_fn': plot,
    'leg_radius': r,
    'actions': actions,
    'target_tolerance_multiplier': 0.5
}

outer_radius = config['max_marks'] * 2 * r + 0.2 * r
bounds = [-outer_radius, outer_radius, -outer_radius, outer_radius]

args = {
    'config': config,
    'dt': 1,
    'bounds': bounds,
    'seq_size': 3,
    'target_phase_steps': 8,
    'heading_phase_steps': 8,
    'radius_multipliers': [1, 2]
}

# Initialize Environment
env = MultiMarkNoModelEnv(**args)

# Initialize PPO model
model = PPO("MlpPolicy", env, seed=0, verbose=1, device='cuda', policy_kwargs={
    "net_arch": [100, 400, 300],
    # "features_extractor_class": SelfAttentionExtractor,
    # "features_extractor_kwargs": {"features_dim": 64}
})

# model.load("BoatControl_boat_model_fine.dat")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [6]:
model.learn(total_timesteps=2_000_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 476      |
|    ep_rew_mean     | -58.4    |
| time/              |          |
|    fps             | 1610     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
Missed mark
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 453         |
|    ep_rew_mean          | -61         |
| time/                   |             |
|    fps                  | 1361        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014146963 |
|    clip_fraction        | 0.13        |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.6        |
|    explained_variance   | 0.0484      |
|    learning_rate

In [7]:
model.save("BoatControl_boat_model_coarse.dat")

In [8]:
# args['config']['target_tolerance_multiplier'] = 3
# model.set_env(MultiMarkNoModelEnv(**args))

In [9]:
model.learn(total_timesteps=2_000_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 353      |
|    ep_rew_mean     | 157      |
| time/              |          |
|    fps             | 1723     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
Missed mark
Missed mark
Missed mark
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 292         |
|    ep_rew_mean          | 151         |
| time/                   |             |
|    fps                  | 1427        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.040142998 |
|    clip_fraction        | 0.203       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.559      |
|    explained_variance   | 0.948   

In [10]:
model.save("BoatControl_boat_model_fine.dat")

In [11]:
def render(data, marks, bounds):
    x_values = [point['x'] for point in data]
    y_values = [point['y'] for point in data]
    meta_values = [{k: '%.3f' % v for (k, v) in point['meta'].items()} for point in data]

    min_x, max_x, min_y, max_y = bounds


    color_meta = [point['meta']['current_mark'] for point in data]
    colors = [colormap[color % len(colormap)] for color in color_meta]

    headings = [point['meta']['heading'] for point in data]
    vmg = [point['meta']['vmg'] for point in data]

    global_meta = {'final_position': (x_values[-1], y_values[-1]), 'reward': sum([point['meta']['reward'] for point in data]), 'iters': len(data), 'episode': episode}

    # Update data
    scatter = fig.data[0]
    scatter.x = x_values
    scatter.y = y_values
    scatter.hovertext = meta_values  # Add this line
    scatter.marker.color = colors
    scatter.line.color = 'lightgrey'

    marks_scatter = fig.data[1]
    marks_scatter.x = [x for x, _ in marks]
    marks_scatter.y = [y for _, y in marks]
    marks_scatter.text = list(range(len(marks)))
    marks_scatter.textposition = 'bottom right'
    marks_scatter.marker.color = 'black'
    marks_scatter.marker.symbol = 'x'
    marks_scatter.marker.size = 10

    heading_scatter = heading_fig.data[0]
    heading_scatter.x = list(range(len(headings)))
    heading_scatter.y = headings
    heading_scatter.marker.color = colors
    heading_scatter.line.color = 'lightgrey'

    vmg_scatter = vmg_fig.data[0]
    vmg_scatter.x = list(range(len(vmg)))
    vmg_scatter.y = vmg
    vmg_scatter.marker.color = colors
    vmg_scatter.line.color = 'lightgrey'

    fig.update_xaxes(range=[min_x * 1.1, max_x * 1.1],dtick=25)
    fig.update_yaxes(range=[min_y * 1.1, max_y * 1.1],dtick=25)
    fig.layout.width=800
    fig.layout.height=800

    heading_fig.update_yaxes(range=[-210, 210],dtick=30)
    heading_fig.layout.width=800
    heading_fig.layout.height=600

    vmg_fig.layout.width=800
    vmg_fig.layout.height=400

    # Update the text output
    with out:
        out.clear_output(wait=True)
        print(f"Global Meta: {str(global_meta)}")


# Initialize output for the text
out = widgets.Output()

# Display text and figure
display(out)
display(fig)
display(heading_fig)
display(vmg_fig)

Output()

FigureWidget({
    'data': [{'hovertext': [{'current_mark': '0.000', 'vmg': '0.000', 'heading':
                            '-135.000', 'reward': '0.000', 'speed': '0.000',
                            'distance': '250.000', 'min_distance': '250.000',
                            'has_missed_mark': '0.000'}, {'current_mark': '0.000',
                            'vmg': '1.269', 'heading': '-136.000', 'reward':
                            '0.635', 'speed': '1.773', 'distance': '248.728',
                            'min_distance': '248.728', 'has_missed_mark': '0.000'},
                            {'current_mark': '0.000', 'vmg': '1.264', 'heading':
                            '-137.000', 'reward': '0.632', 'speed': '1.745',
                            'distance': '247.461', 'min_distance': '247.461',
                            'has_missed_mark': '0.000'}, {'current_mark': '0.000',
                            'vmg': '0.491', 'heading': '-142.000', 'reward':
                            '0.

FigureWidget({
    'data': [{'line': {'color': 'lightgrey'},
              'marker': {'color': [blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                          

FigureWidget({
    'data': [{'line': {'color': 'lightgrey'},
              'marker': {'color': [blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                          

In [12]:
target_x = [0, 0]
target_y = [125, 250]

best_trajectory = None
best_reward = None

env = MultiMarkNoModelEnv(**args)

model = PPO.load("BoatControl_boat_model_fine.dat")

for target_x, target_y in zip(target_x, target_y):
    for i in range(20):
        obs, _ = env.reset()
        env.target_x[0] = target_x
        env.target_y[0] = target_y
        env.heading = 0
        done = False
        total_reward = 0
        while not done:
            action, _states = model.predict(obs)
            obs, reward, is_terminal, is_truncated, info = env.step(action)
            done = is_terminal or is_truncated
            total_reward += reward

        if best_reward is None or total_reward > best_reward:
            best_reward = total_reward
            best_trajectory = env.trajectory

    render(best_trajectory, [(target_x, target_y)], bounds)

Missed mark
Missed mark
Missed mark
